In [ ]:
import os
import random
import glob
import pandas as pd
import numpy as np
import pydicom 


import torch
import torch.optim as optim
from albumentations import Compose, ShiftScaleRotate, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset,Subset

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, jaccard_score, confusion_matrix
from tqdm import notebook as tqdm

In [ ]:
INPUT_PATH = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'

In [ ]:
os.listdir(INPUT_PATH)

In [ ]:
# Parameters
n_classes = 6
n_epochs = 1
batch_size = 32

In [ ]:
COLS = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']

In [ ]:
train_images_dir = os.path.join(INPUT_PATH, 'stage_2_train/')
test_images_dir = os.path.join(INPUT_PATH, 'stage_2_test/')

In [ ]:
# train_metadata_csv = '../input/rsna-stage-2-metadata-ihd-2019/stage_2_train_with_metadata.csv'
# test_metadata_csv = '../input/rsna-stage-2-metadata-ihd-2019/stage_2_test_with_metadata.csv'
train_metadata_csv = '../input/rsna-intracranial-sequence-metadata/train_metadata_noidx.csv'
test_metadata_csv = '../input/rsna-intracranial-sequence-metadata/test_metadata_noidx.csv'

In [ ]:
train_metadata = pd.read_csv(train_metadata_csv)
train_metadata.columns

In [ ]:
train = pd.read_csv(os.path.join(INPUT_PATH, 'stage_2_train.csv'))
test = pd.read_csv(os.path.join(INPUT_PATH, 'stage_2_sample_submission.csv'))

## Preprocessing : Windowing

In [ ]:
## A function to correct pixel data and rescale intercercepts ob 12 bit images
def dcm_correction(dcm_img):
        x = dcm_img.pixel_array + 1000
        px_mode = 4096
        x[x >= px_mode] = x[x >= px_mode] - px_mode #if there are extra bits in 12-bit grayscale(<=4096)
        dcm_img.PixelData = x.tobytes()
        dcm_img.RescaleIntercept = -1000 #setting a common value across all 12-bit US images
        
#Systemic/linear windowing
def window_image(dcm, window_center, window_width):
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        dcm_correction(dcm)

    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept #reconstructing the image from pixels
    img_min = window_center - window_width // 2 #lowest visible value
    img_max = window_center + window_width // 2 #highest visible value
    img = np.clip(img, img_min, img_max)

    return img

#Combining all
def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)

    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1, 2, 0)

    return bsb_img

In [ ]:
class IntracranialDataset(Dataset):

    def __init__(self, csv_file, path, labels, transform=None):
        self.path = path
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
#         img_name = os.path.join(self.path, self.data.loc[idx, 'Image'] + '.png')
#         img = cv2.imread(img_name)   
        try:
            dicom = pydicom.dcmread(self.path, self.data.loc[idx, 'Image'] + '.dcm')
            img = bsb_window(dicom)
        except:
            img = np.zeros((512, 512, 3))
        
        if self.transform:       
            augmented = self.transform(image=img)
            img = augmented['image']   
            
        if self.labels:
            
            labels = torch.tensor(
                self.data.loc[idx, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']])
            return {'image': img, 'labels': labels}    
        
        else:      
            
            return {'image': img}

In [ ]:
train.value_counts()

In [ ]:
train_metadata = pd.read_csv(train_metadata_csv)
test_metadata = pd.read_csv(test_metadata_csv)

In [ ]:
train_metadata[['ID', 'Image', 'Diagnosis']] = train_metadata['ID'].str.split('_', expand=True)
train_metadata['ImageID'] = 'ID_' + train_metadata['Image']
train_metadata.head()

In [ ]:
train_metadata.drop(['ID', 'Diagnosis'], axis=1)

In [ ]:
train_metadata[:15]

## Data Preparation

In [ ]:
# Prepare train table
train[['ID', 'Image', 'Diagnosis']] = train['ID'].str.split('_', expand=True)
train = train[['Image', 'Diagnosis', 'Label']]
train.drop_duplicates(inplace=True)
train = train.pivot(index='Image', columns='Diagnosis', values='Label').reset_index()
train['Image'] = 'ID_' + train['Image']

In [ ]:
# Remove invalid instances of images(dcm, PNG)
png = glob.glob(os.path.join(train_images_dir, '*.dcm'))
png = [os.path.basename(png)[:-4] for png in png]
png = np.array(png)

In [ ]:
train = train[train['Image'].isin(png)]
# train.to_csv('train.csv', index=False)
train.value_counts()

In [ ]:
merged_train = pd.merge(left=train, right=train_metadata, how='left', left_on='Image', right_on='ImageId')

In [ ]:
merged_train

In [ ]:
#Train/valid split 
train_series = train_metadata['SeriesInstanceUID'].unique() #Identifying unique scans by SeriesID: there are totally 

valid_series = train_series[21000:]
train_series = train_series[:21000]

In [ ]:
print(len(train_series))
print(len(valid_series))

In [ ]:
train_series

In [ ]:
train_df = merged_train[merged_train['SeriesInstanceUID'].isin(train_series)]
train_df

In [ ]:
valid_df = merged_train[merged_train['SeriesInstanceUID'].isin(valid_series)]
valid_df

In [ ]:
print(len(train_df))
print(len(valid_df))

So the training set has 728,513 slices and a validation set 24,290 slices

In [ ]:
os.mkdir('./data')

In [ ]:
train_df.to_csv('data/train.csv', index=False)
print(train_df['any'].value_counts())
valid_df.to_csv('data/valid.csv', index=False)
print(valid_df['any'].value_counts())

In [ ]:
# Prepare test table
test[['ID', 'Image', 'Diagnosis']] = test['ID'].str.split('_', expand=True)
test['Image'] = 'ID_' + test['Image']
test = test[['Image', 'Label']]
test.drop_duplicates(inplace=True)

test.to_csv('data/test.csv', index=False)

In [ ]:
# Data loaders
transform_train = Compose([Resize(256, 256),
                           Normalize(mean=[0.1738, 0.1433, 0.1970], std=[0.3161, 0.2850, 0.3111], max_pixel_value=1.),
                           HorizontalFlip(),
                           ShiftScaleRotate(),
                           RandomBrightnessContrast(),
                           ToTensorV2()])

transform_test = Compose([Resize(256, 256),
                          Normalize(mean=[0.1738, 0.1433, 0.1970], std=[0.3161, 0.2850, 0.3111], max_pixel_value=1.),
                          ToTensorV2()])


In [ ]:
train_dataset = IntracranialDataset(
    csv_file='data/train.csv', path=train_images_dir, transform=transform_train, labels=True)
# print(len(train_dataset))

valid_dataset = IntracranialDataset(
    csv_file='data/valid.csv', path=train_images_dir, transform=transform_train, labels=True)
# print(len(valid_dataset))



In [ ]:
test_dataset = IntracranialDataset(
    csv_file='data/test.csv', path=test_images_dir, transform=transform_test, labels=False)
# print(len(test_dataset))

In [ ]:
data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
print(len(data_loader_train))
data_loader_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
print(len(data_loader_valid))

In [ ]:
data_loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
print(len(data_loader_test))

## Model

In [ ]:
from torch.hub import load_state_dict_from_url
from torchvision.models.resnet import ResNet, Bottleneck

In [ ]:
model_urls = {
    'resnext101_32x8d': 'https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth',
    'resnext101_32x16d': 'https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth',
    'resnext101_32x32d': 'https://download.pytorch.org/models/ig_resnext101_32x32-e4b90b00.pth',
    'resnext101_32x48d': 'https://download.pytorch.org/models/ig_resnext101_32x48-3e41cc8a.pth',
}

In [ ]:
def _resnext(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
    model.load_state_dict(state_dict)
    return model

def resnext101_32x8d_wsl(progress=True, **kwargs):
    """Constructs a ResNeXt-101 32x8 model pre-trained on weakly-supervised data
    and finetuned on ImageNet from Figure 5 in
    `"Exploring the Limits of Weakly Supervised Pretraining" <https://arxiv.org/abs/1805.00932>`_
    Args:
        progress (bool): If True, displays a progress bar of the download to stderr.
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnext('resnext101_32x8d', Bottleneck, [3, 4, 23, 3], True, progress, **kwargs)

In [ ]:
model = resnext101_32x8d_wsl()
print(model)

In [ ]:
list(model.children())

In [ ]:
class ResNeXtModel(torch.nn.Module):
    def __init__(self):
        super(ResNeXtModel, self).__init__()
        resnext = resnext101_32x8d_wsl()
        self.base = torch.nn.Sequential(*list(resnext.children())[:-1])
        self.fc = torch.nn.Sequential(torch.nn.Linear(2048,6))
    
    def forward(self, input):
        features = self.base(input).reshape(-1, 2048)
        out = self.fc(features)
        return out, features

## Training & Validation

In [ ]:
# Installing useful libraries

!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
#For Mixed precision training
from apex import amp

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = ResNeXtModel()
model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

In [ ]:
!pip install GPUtil

In [ ]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

In [ ]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

In [ ]:
torch.cuda.empty_cache()

In [ ]:
gpu_usage()  

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
tb  = SummaryWriter('runs/ich_detection_experiment_1')
# tb  = SummaryWriter()

In [ ]:
for epoch in range(n_epochs):

    print('Epoch {}/{}'.format(epoch + 1, n_epochs))
    print('-' * 10)

    model.train()
    tr_loss = 0
    tr_correct = 0
    
    tk0 = tqdm.tqdm(data_loader_train, desc="Iteration")

    for step, batch in enumerate(tk0):
        inputs = batch["image"]
#         print(inputs.shape)
        labels = batch["labels"]
#         print(labels.shape)

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs, _ = model(inputs)
#         print(outputs.shape)
#         print(outputs)
        loss = criterion(outputs, labels)
        preds = (torch.sigmoid(outputs) >=0.5).float()*1
#         print(preds.shape)


        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
            
        tr_loss += loss.item()
        tr_correct += torch.sum(preds == labels)
#         print(tr_correct)

        optimizer.step()
        optimizer.zero_grad()

        if step % 512 == 0:
            epoch_loss = tr_loss / (step + 1)
            print('Training Loss at {}: {:.4f}'.format(step, epoch_loss))

    epoch_loss = tr_loss / len(data_loader_train)
    print('Training Loss: {:.4f}'.format(epoch_loss))
    print('-----------------------')
    #Tensorboard code for visualisations
    tb.add_scalar("Training Loss", tr_loss, epoch)
    tb.add_scalar("Training Correct preds", tr_correct, epoch)
    tb.add_scalar("Training Accuracy", tr_correct/ len(train_dataset), epoch)
    print('Finished Training!')
    
    model.eval()
    tr_loss = 0
    tr_correct = 0

    auc_preds = []
    auc_truths = []
    
    print('Validation starts...')
    for step, batch in enumerate(data_loader_valid):
        inputs = batch["image"]
        labels = batch["labels"]

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs, _ = model(inputs)
        loss = criterion(outputs, labels)
        preds = (torch.sigmoid(outputs) >=0.5).float()*1

        tr_loss += loss.item()
        tr_correct += torch.sum(preds == labels)

        auc_preds.append(preds.view(-1, 6).detach().cpu().numpy())
        auc_truths.append(labels.view(-1, 6).detach().cpu().numpy())

    epoch_loss = tr_loss / len(data_loader_valid)
    print('Validation Loss: {:.4f}'.format(epoch_loss))
    
    #Tensorboard code for visualisations
    tb.add_scalar("Validation Loss", tr_loss, epoch)
    tb.add_scalar("Training Correct preds", tr_correct, epoch)
    tb.add_scalar("Training Accuracy", tr_correct/ len(train_dataset), epoch)
    print('Finished Validation!')
    
    roc_preds = np.concatenate(auc_preds)

    roc_truths = np.concatenate(auc_truths)

    for tp in range(0, 6):
        print('ROC_AUC')
        print(COLS[tp], roc_auc_score(roc_truths[:, tp], roc_preds[:, tp]), )
        print('F1 SCORE')
        print(COLS[tp], f1_score(roc_truths[:, tp], roc_preds[:, tp]), )
        print('ACCURACY')
        print(COLS[tp], accuracy_score(roc_truths[:, tp], roc_preds[:, tp]), )
        tn, fp, fn, tp = confusion_matrix(roc_truths[:, tp], roc_preds[:, tp]).ravel()
        print('SENSITIVITY')
        print(COLS[tp], tp/(tp+fn), )
        print('SPECIFICITY')
        print(COLS[tp], tn/(tn+fp), )    
        print('JACCARD SCORE')
        print(COLS[tp], jaccard_score(roc_truths[:, tp], roc_preds[:, tp]), )
        
    print('-----------------------')

# Save checkpoint
checkpoint = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    # 'amp': amp.state_dict()
}
torch.save(checkpoint, 'model.pt')
tb.close()

**Points to note -**
- It is **NOT** multi-class but a **multi-label** classification problem(Refer to the paper)
- Try using Sigmoid function to get 0or1 across each class, NOT Softmax
- While finding accuracy, to fin correct predictions use threshold for outputs as 0.5

**Observations and points to noye** -
- On just 1 epoch, the results are **very poor**
- Accuracy is high for obvious reasons. It **should not be used** to judge, since it is an **imbalaced problem**
- F1 should be used when **we care about positive classes**. We should look at improving this
- ROC-AUC should be used when you care **equally about positive and negative classes**. **Not advised on highly imbalanced** dataset(but let's check what we get and comapre with paper. Look into *average_precision_score* metric instead(when you care more about positive than negative class)
- What did we get?
    - Accuracy high - DO NOT go by this
    - F1 of 0 - very bad, hope it improves when trained on more epochs
    - ROC-AUC of 0.5 - which means not discriminating between the two claases(0/1); again hope to improve
    
**NOTE**
Error fixed. Should work tommorrow(::finger crossed::)
- Tensorboard visualisation codes are right! But the site to display them doesn't seem to work through kaggle. It should work locally

In [ ]:
tb.close()

In [ ]:
!tensorboard --logdir=runs

## Testing

In [ ]:
# Nothing yet